In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"   #(xxxx is your specific GPU ID)

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Activation, Dense, Flatten, GlobalAveragePooling2D
from MyEarlyStopping import MyEarlyStopping
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder  
import pandas as pd
import timeit

In [3]:
All_data = ImageDataGenerator(preprocessing_function=preprocess_input)
All_data_generator = All_data.flow_from_directory('/data/home/cou/yuli5/Audio/online_data_adaption/Mel_Audio_folder_digits',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 30000 images belonging to 10 classes.


In [4]:
image_names = All_data_generator.filenames
image_no = [i.split("/")[1].split("_")[2].split(".")[0] for i in image_names]
image_no = np.array(list(map(int, image_no)))
ALL_participant_class = [i.split("/")[1].split("_")[1] for i in image_names]
ALL_participant_class = np.array(list(map(int, ALL_participant_class)))
command_class = All_data_generator.classes
All_participant_class = tf.keras.utils.to_categorical(ALL_participant_class-1, num_classes=60)
All_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)
All_command_uniform = All_command_class*0+1/10

In [5]:
All_Inputs = [next(All_data_generator)[0][0] for _ in range(len(All_data_generator))]
All_Inputs = np.array(All_Inputs)

In [6]:
# image_no<=20 means every word repeat 20 times 
s_number = 5 # ALL_participant_class  the subject index

select_indexs_train = (image_no<=19)&(ALL_participant_class<=s_number)
Train_Inputs = All_Inputs[select_indexs_train]
Train_participant_class = All_participant_class[select_indexs_train][:,0:s_number]
Train_participant_uniform = Train_participant_class*0+1/s_number
Train_command_class = All_command_class[select_indexs_train]
Train_command_uniform = Train_command_class*0+1/10

In [7]:
select_indexs_val = (image_no>29)&(image_no<=39)&(ALL_participant_class<=s_number)
Val_Inputs = All_Inputs[select_indexs_val]
Val_participant_class = All_participant_class[select_indexs_val][:,0:s_number]
Val_participant_uniform = Val_participant_class*0+1/s_number
Val_command_class = All_command_class[select_indexs_val]
Val_command_uniform = Val_command_class*0+1/10

In [8]:
select_indexs_test = (image_no>39)&(image_no<=49)&(ALL_participant_class<=s_number)
Test_Inputs = All_Inputs[select_indexs_test]
Test_participant_class = All_participant_class[select_indexs_test][:,0:s_number]
Test_participant_uniform = Test_participant_class*0+1/s_number
Test_command_class = All_command_class[select_indexs_test]
Test_command_uniform = Test_command_class*0+1/10

In [9]:
Perfomance = pd.read_csv('Performance_0915_Digit_Inputs.csv')

In [10]:
actv_fun_1_1 = "relu" 
actv_fun_1_2 = "sigmoid"
shape_1_1 = 128
shape_1_2 = 256
actv_fun_2_1 = "sigmoid" 
actv_fun_2_2 = "sigmoid"
shape_2_1 = 512
shape_2_2 = 512


In [11]:
high_acc = 0
for run in range(0,10):
    
    # feature extraction layers
    resnet_model = VGG16(input_shape=(224, 224,3), include_top=False, weights="imagenet")
    for layer in resnet_model.layers:
        layer.trainable = False

    d = resnet_model.output.shape[-1] # dimension of last layer

    ###################### model 1 ###################### 
    layer_1_0 = tf.keras.layers.Dense(d,name="weight_1")(resnet_model.output) #times weight before flatten
    layer_1_1 = tf.keras.layers.Flatten(name='flatten_1')(layer_1_0)

    Dense_1_1 = tf.keras.layers.Dense(shape_1_1, activation=actv_fun_1_1,name='fc1_1')
    layer_1_2 = Dense_1_1(layer_1_1)
    Dense_1_2 = tf.keras.layers.Dense(shape_1_2, activation=actv_fun_1_2,name='fc1_2')
    layer_1_3 = Dense_1_2(layer_1_2)

    Dense_1_3 = tf.keras.layers.Dense(s_number, activation='softmax' ,name='participant_output')
    out_layer_1 = Dense_1_3(layer_1_3)

    ###################### model 2 ###################### 
    layer_2_0 = tf.keras.layers.Dense(d,name="weight_2")(resnet_model.output) #times weight before flatten
    layer_2_1 = tf.keras.layers.Flatten(name='flatten_2')(layer_2_0)

    Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
    layer_2_2  = Dense_2_1(layer_2_1)
    Dense_2_2 = tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
    layer_2_3  = Dense_2_2(layer_2_2)

    Dense_2_3 = tf.keras.layers.Dense(10, activation='softmax',name='command_output')
    out_layer_2 = Dense_2_3(layer_2_3)

    ###################### model 1' ###################### 
    layer_1_2_  = Dense_2_1(layer_1_1)
    layer_1_3_  = Dense_2_2(layer_1_2_)
    out_layer_1_ = Dense_2_3(layer_1_3_)

    ###################### model 1' ###################### 
    layer_2_2_  = Dense_1_1(layer_2_1)
    layer_2_3_  = Dense_1_2(layer_2_2_)
    out_layer_2_ = Dense_1_3(layer_2_3_)

    resnet_model = tf.keras.Model(resnet_model.input, [out_layer_1,out_layer_2,out_layer_1_,out_layer_2_])

    # resnet_model.summary() 

    w_1, w_2, w_1_, w_2_ = 1,1,1,1
    ##################### training ############################
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    callbacks = [MyEarlyStopping(monitor1 = 'val_' + resnet_model.layers[-1].name+'_accuracy',
                                  monitor2 = 'val_' + resnet_model.layers[-2].name+'_accuracy',
                                  patience=5,restore_best_weights=True)]
    resnet_model.compile(optimizer=opt, loss=["categorical_crossentropy","categorical_crossentropy","mse","mse"],
                         loss_weights=[w_1, w_2, w_1_, w_2_], metrics=['accuracy'])
    start = timeit.default_timer()
    history1 = resnet_model.fit(Train_Inputs, 
                               {resnet_model.layers[-2].name:Train_participant_class, 
                                resnet_model.layers[-1].name:Train_command_class,
                                resnet_model.layers[-1].name+"_1":Train_command_uniform, 
                                resnet_model.layers[-2].name+"_1":Train_participant_uniform}, 
                                validation_data=(Val_Inputs,
                                                 {resnet_model.layers[-2].name:Val_participant_class,
                                                  resnet_model.layers[-1].name:Val_command_class,
                                                  resnet_model.layers[-1].name+"_1":Val_command_uniform,
                                                  resnet_model.layers[-2].name+"_1":Val_participant_uniform}), 
                               callbacks=callbacks,
                               batch_size=64,
                               epochs=10)

    for layer in resnet_model.layers[0:19]:
        layer.trainable = True
    history2 = resnet_model.fit(Train_Inputs, 
                               {resnet_model.layers[-2].name:Train_participant_class, 
                                resnet_model.layers[-1].name:Train_command_class,
                                resnet_model.layers[-1].name+"_1":Train_command_uniform, 
                                resnet_model.layers[-2].name+"_1":Train_participant_uniform}, 
                                validation_data=(Val_Inputs,
                                                 {resnet_model.layers[-2].name:Val_participant_class,
                                                  resnet_model.layers[-1].name:Val_command_class,
                                                  resnet_model.layers[-1].name+"_1":Val_command_uniform,
                                                  resnet_model.layers[-2].name+"_1":Val_participant_uniform}), 
                               callbacks=callbacks,
                               batch_size=64,
                               epochs=10)
   
    stop = timeit.default_timer()
    run_time = stop - start
    
    predictions = resnet_model.predict(Test_Inputs)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    participant_class = [i.split("/")[1].split("_")[1] for i in All_data_generator.filenames]
    participant_class = np.array(participant_class)[select_indexs_test]
    participant_class = np.array(list(map(int, participant_class)))
    acc_s = round(sum(x == y for x, y in zip(participant_class, predicted_classes)) / len(participant_class),4)
    
    predictions = resnet_model.predict(Test_Inputs)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.array([i.split("/")[0] for i in All_data_generator.filenames])
    true_classes = np.array(list(map(int, true_classes)))[select_indexs_test]
    acc_c = round(sum(x == y for x, y in zip(true_classes, predicted_classes)) / len(true_classes),4)
    overall_acc = acc_c + acc_s

    Perfomance = Perfomance.append({'Feature_Extract': "VGG16",'Inputs': "MEL",'Time':run_time,
                                    'Partcp_Acc':acc_s,'Command_Acc':acc_c}, ignore_index=True)
    
    if high_acc < overall_acc:
        resnet_model.save('Digital_data_Initial_group_model_0915_vgg.h5')
        high_acc = overall_acc
        best_index = run
        
    del resnet_model
    run = run + 1

Epoch 1/10
16/16 [==============================] - 4s 239ms/step - loss: 3.6688 - participant_output_loss: 1.5492 - command_output_loss: 2.1145 - command_output_1_loss: 8.6479e-04 - participant_output_1_loss: 0.0043 - participant_output_accuracy: 0.3790 - command_output_accuracy: 0.4310 - command_output_1_accuracy: 0.2140 - participant_output_1_accuracy: 0.2060 - val_loss: 3.1929 - val_participant_output_loss: 1.2790 - val_command_output_loss: 1.9131 - val_command_output_1_loss: 1.6449e-04 - val_participant_output_1_loss: 5.5505e-04 - val_participant_output_accuracy: 0.7180 - val_command_output_accuracy: 0.7140 - val_command_output_1_accuracy: 0.0020 - val_participant_output_1_accuracy: 0.3480
Epoch 2/10
16/16 [==============================] - 2s 128ms/step - loss: 2.9761 - participant_output_loss: 1.1545 - command_output_loss: 1.8211 - command_output_1_loss: 1.1662e-04 - participant_output_1_loss: 3.6478e-04 - participant_output_accuracy: 0.7490 - command_output_accuracy: 0.8230 - c

Epoch 3/10
16/16 [==============================] - 2s 143ms/step - loss: 0.5562 - participant_output_loss: 0.1585 - command_output_loss: 0.3976 - command_output_1_loss: 5.6237e-05 - participant_output_1_loss: 1.1434e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9990 - command_output_1_accuracy: 0.1020 - participant_output_1_accuracy: 0.3970 - val_loss: 0.6802 - val_participant_output_loss: 0.2523 - val_command_output_loss: 0.4278 - val_command_output_1_loss: 4.9480e-05 - val_participant_output_1_loss: 1.3717e-05 - val_participant_output_accuracy: 0.9700 - val_command_output_accuracy: 0.9920 - val_command_output_1_accuracy: 0.0860 - val_participant_output_1_accuracy: 0.4020
Epoch 4/10
16/16 [==============================] - 2s 121ms/step - loss: 0.4898 - participant_output_loss: 0.1446 - command_output_loss: 0.3452 - command_output_1_loss: 4.6619e-05 - participant_output_1_loss: 9.9048e-06 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9990

Epoch 5/10
16/16 [==============================] - 3s 160ms/step - loss: 1.8941 - participant_output_loss: 0.3980 - command_output_loss: 1.4958 - command_output_1_loss: 5.2866e-05 - participant_output_1_loss: 1.9319e-04 - participant_output_accuracy: 0.9770 - command_output_accuracy: 0.9830 - command_output_1_accuracy: 0.0310 - participant_output_1_accuracy: 0.2660 - val_loss: 1.8776 - val_participant_output_loss: 0.4364 - val_command_output_loss: 1.4410 - val_command_output_1_loss: 5.2318e-05 - val_participant_output_1_loss: 1.9771e-04 - val_participant_output_accuracy: 0.9500 - val_command_output_accuracy: 0.9620 - val_command_output_1_accuracy: 0.0120 - val_participant_output_1_accuracy: 0.2620
Epoch 6/10
16/16 [==============================] - 3s 166ms/step - loss: 1.6863 - participant_output_loss: 0.3198 - command_output_loss: 1.3663 - command_output_1_loss: 5.5798e-05 - participant_output_1_loss: 1.7005e-04 - participant_output_accuracy: 0.9920 - command_output_accuracy: 0.9920

Epoch 7/10
16/16 [==============================] - 2s 140ms/step - loss: 0.4539 - participant_output_loss: 0.0687 - command_output_loss: 0.3851 - command_output_1_loss: 3.8055e-05 - participant_output_1_loss: 4.5978e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.0540 - participant_output_1_accuracy: 0.3180 - val_loss: 0.5723 - val_participant_output_loss: 0.1549 - val_command_output_loss: 0.4173 - val_command_output_1_loss: 3.1605e-05 - val_participant_output_1_loss: 5.3782e-05 - val_participant_output_accuracy: 0.9760 - val_command_output_accuracy: 0.9940 - val_command_output_1_accuracy: 0.0300 - val_participant_output_1_accuracy: 0.3200
Epoch 8/10
16/16 [==============================] - 2s 152ms/step - loss: 0.4078 - participant_output_loss: 0.0632 - command_output_loss: 0.3445 - command_output_1_loss: 3.2824e-05 - participant_output_1_loss: 3.9832e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000

Epoch 10/10
16/16 [==============================] - 2s 132ms/step - loss: 0.8372 - participant_output_loss: 0.1289 - command_output_loss: 0.7081 - command_output_1_loss: 6.9340e-05 - participant_output_1_loss: 1.2094e-04 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9970 - command_output_1_accuracy: 0.1790 - participant_output_1_accuracy: 0.2170 - val_loss: 0.9128 - val_participant_output_loss: 0.2031 - val_command_output_loss: 0.7096 - val_command_output_1_loss: 5.6881e-05 - val_participant_output_1_loss: 1.5208e-04 - val_participant_output_accuracy: 0.9580 - val_command_output_accuracy: 0.9900 - val_command_output_1_accuracy: 0.1520 - val_participant_output_1_accuracy: 0.2560
Epoch 1/10
16/16 [==============================] - 2s 143ms/step - loss: 0.7216 - participant_output_loss: 0.1055 - command_output_loss: 0.6158 - command_output_1_loss: 6.8865e-05 - participant_output_1_loss: 1.0794e-04 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.998

...

Epoch 6/10
16/16 [==============================] - 2s 131ms/step - loss: 1.6027 - participant_output_loss: 0.4083 - command_output_loss: 1.1942 - command_output_1_loss: 9.1348e-05 - participant_output_1_loss: 1.0589e-04 - participant_output_accuracy: 0.9870 - command_output_accuracy: 0.9950 - command_output_1_accuracy: 0.0350 - participant_output_1_accuracy: 0.1070 - val_loss: 1.6064 - val_participant_output_loss: 0.4601 - val_command_output_loss: 1.1461 - val_command_output_1_loss: 8.2492e-05 - val_participant_output_1_loss: 1.0433e-04 - val_participant_output_accuracy: 0.9540 - val_command_output_accuracy: 0.9740 - val_command_output_1_accuracy: 0.0340 - val_participant_output_1_accuracy: 0.1360
Epoch 7/10
16/16 [==============================] - 2s 135ms/step - loss: 1.3961 - participant_output_loss: 0.3469 - command_output_loss: 1.0490 - command_output_1_loss: 7.7612e-05 - participant_output_1_loss: 8.5497e-05 - participant_output_accuracy: 0.9920 - command_output_accuracy: 0.9980

Epoch 8/10
16/16 [==============================] - 2s 123ms/step - loss: 0.3380 - participant_output_loss: 0.0885 - command_output_loss: 0.2494 - command_output_1_loss: 3.3545e-05 - participant_output_1_loss: 1.0275e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.1310 - participant_output_1_accuracy: 0.0250 - val_loss: 0.4737 - val_participant_output_loss: 0.1767 - val_command_output_loss: 0.2969 - val_command_output_1_loss: 3.2122e-05 - val_participant_output_1_loss: 1.3114e-05 - val_participant_output_accuracy: 0.9660 - val_command_output_accuracy: 0.9900 - val_command_output_1_accuracy: 0.1800 - val_participant_output_1_accuracy: 0.0380
Epoch 9/10
16/16 [==============================] - 2s 127ms/step - loss: 0.3057 - participant_output_loss: 0.0813 - command_output_loss: 0.2244 - command_output_1_loss: 2.9728e-05 - participant_output_1_loss: 9.8808e-06 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000

Epoch 10/10
16/16 [==============================] - 2s 124ms/step - loss: 0.7855 - participant_output_loss: 0.2259 - command_output_loss: 0.5594 - command_output_1_loss: 8.5523e-05 - participant_output_1_loss: 7.6265e-05 - participant_output_accuracy: 0.9980 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.1220 - participant_output_1_accuracy: 0.1920 - val_loss: 0.8809 - val_participant_output_loss: 0.3076 - val_command_output_loss: 0.5731 - val_command_output_1_loss: 7.4429e-05 - val_participant_output_1_loss: 9.2523e-05 - val_participant_output_accuracy: 0.9640 - val_command_output_accuracy: 0.9900 - val_command_output_1_accuracy: 0.0540 - val_participant_output_1_accuracy: 0.2460
Epoch 1/10
16/16 [==============================] - 2s 141ms/step - loss: 0.6794 - participant_output_loss: 0.2012 - command_output_loss: 0.4781 - command_output_1_loss: 8.2502e-05 - participant_output_1_loss: 7.1085e-05 - participant_output_accuracy: 0.9990 - command_output_accuracy: 1.000

Epoch 2/10
16/16 [==============================] - 2s 125ms/step - loss: 2.8431 - participant_output_loss: 0.9839 - command_output_loss: 1.8585 - command_output_1_loss: 1.0563e-04 - participant_output_1_loss: 6.0233e-04 - participant_output_accuracy: 0.7820 - command_output_accuracy: 0.8810 - command_output_1_accuracy: 0.0200 - participant_output_1_accuracy: 0.2640 - val_loss: 2.6788 - val_participant_output_loss: 0.9148 - val_command_output_loss: 1.7635 - val_command_output_1_loss: 5.3362e-05 - val_participant_output_1_loss: 4.3806e-04 - val_participant_output_accuracy: 0.7720 - val_command_output_accuracy: 0.9040 - val_command_output_1_accuracy: 0.0200 - val_participant_output_1_accuracy: 0.1520
Epoch 3/10
16/16 [==============================] - 2s 141ms/step - loss: 2.3963 - participant_output_loss: 0.7116 - command_output_loss: 1.6843 - command_output_1_loss: 7.9726e-05 - participant_output_1_loss: 3.0527e-04 - participant_output_accuracy: 0.9050 - command_output_accuracy: 0.9170

Epoch 4/10
16/16 [==============================] - 2s 149ms/step - loss: 0.5425 - participant_output_loss: 0.0796 - command_output_loss: 0.4628 - command_output_1_loss: 8.7310e-05 - participant_output_1_loss: 5.5767e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.0460 - participant_output_1_accuracy: 0.1130 - val_loss: 0.6527 - val_participant_output_loss: 0.1611 - val_command_output_loss: 0.4915 - val_command_output_1_loss: 7.5506e-05 - val_participant_output_1_loss: 6.5743e-05 - val_participant_output_accuracy: 0.9780 - val_command_output_accuracy: 0.9900 - val_command_output_1_accuracy: 0.2100 - val_participant_output_1_accuracy: 0.1060
Epoch 5/10
16/16 [==============================] - 2s 135ms/step - loss: 0.4820 - participant_output_loss: 0.0711 - command_output_loss: 0.4108 - command_output_1_loss: 6.4204e-05 - participant_output_1_loss: 4.8357e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000

In [12]:
Perfomance

,Command_Acc,Feature_Extract,Inputs,Partcp_Acc,Time
0,0.982,ResNet,SPEC,0.970,53.745848
1,0.980,ResNet,SPEC,0.968,59.788192
2,0.972,ResNet,SPEC,0.960,60.039808
3,0.982,ResNet,SPEC,0.958,56.502710
4,0.980,ResNet,SPEC,0.968,56.194534
5,0.986,ResNet,SPEC,0.966,55.374021
6,0.978,ResNet,SPEC,0.952,55.079770
7,0.988,ResNet,SPEC,0.960,55.301056
8,0.984,ResNet,SPEC,0.960,51.448608
9,0.980,ResNet,SPEC,0.952,52.307214


In [13]:
Perfomance.to_csv('Performance_0915_Digit_Inputs.csv',index=False)